In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
import json
import os
import re
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import pandas as pd
import numpy as np
import json
import re
import torch

# Data Cleaning

In [5]:
# Data Path and Processed data saving path

data_path = "/content/drive/MyDrive/CSE4095 - NLP/Datasets/raw/2021-01"
output_saving_path = "/content/drive/MyDrive/CSE4095 - NLP/Datasets/processed"

In [6]:
# Regular Expression to find all white spaces
pattern = re.compile(r'\s+')

In [8]:
# listing all files
list_of_files = os.listdir(data_path)

### Cleaning Unnecessary White Spaces

In [13]:
# An example
with open(f"{data_path}/{list_of_files[0]}", 'rb') as f:
    an_instance = json.load(f)

In [15]:
# Original Ictihat
an_instance['ictihat']

'   HÜKÜM : Mahkumiyet    Mahalli mahkemece verilen hüküm temyiz edilmekle evrak okunarak;  Gereği görüşülüp düşünüldü:  Gereği görüşülüp düşünüldü;  Yapılan yargılamaya, toplanan ve karar yerinde açıklanan delillere, mahkemenin kovuşturma sonucunda oluşan inanç ve takdirine, gösterilen gerekçeye ve uygulamaya göre sanık müdafiinin yerinde görülmeyen temyiz sebeplerinin reddiyle hükmün ONANMASINA, 14.01.2021 gününde oy birliğiyle karar verildi.'

In [16]:
# Cleaning ictihats with replacing all white spaces with a single space
re.sub(pattern, ' ', an_instance['ictihat']).strip()

'HÜKÜM : Mahkumiyet Mahalli mahkemece verilen hüküm temyiz edilmekle evrak okunarak; Gereği görüşülüp düşünüldü: Gereği görüşülüp düşünüldü; Yapılan yargılamaya, toplanan ve karar yerinde açıklanan delillere, mahkemenin kovuşturma sonucunda oluşan inanç ve takdirine, gösterilen gerekçeye ve uygulamaya göre sanık müdafiinin yerinde görülmeyen temyiz sebeplerinin reddiyle hükmün ONANMASINA, 14.01.2021 gününde oy birliğiyle karar verildi.'

In [9]:
# Showing duplicate files
for i in list_of_files:
    if '(' in i:
        print(i)

9395(1).json
9400(1).json
9397(1).json
9396(1).json
9394(1).json
9399(1).json
9398(1).json
940(1).json
94(1).json


In [10]:
# Sorting file names to preserve the order
sorted_list_of_files = sorted([int(i.split('.')[0]) for i in list_of_files if '(' not in i])
sorted_list_of_files = [f"{i}.json" for i in sorted_list_of_files]

In [11]:
# Reading ictihats from files
def read_files(file_name):
    with open(f"{data_path}/{file_name}", 'rb') as f:
        an_instance = json.load(f)

    an_instance['ictihat'] = re.sub(pattern, ' ', an_instance['ictihat']).strip()
    return an_instance

In [18]:
# Reading files using multi-threading
with ThreadPoolExecutor() as executor:
    cleaned_jsons = list(executor.map(read_files, sorted_list_of_files))

In [ ]:
with open(f"{output_saving_path}/cleaned_and_combined_data.json", 'w', encoding='utf-8') as f:
    json.dump(cleaned_jsons, f, ensure_ascii=False, indent=4)

# Tagging

In [20]:
# Reading data again if notebook restarted
with open(f"{output_saving_path}/cleaned_and_combined_data.json", 'r', encoding='utf-8') as f:
    cleaned_jsons = json.load(f)

In [27]:
# Finding Starting and Ending Indices of Kanuns and Ust Mahkemes
def find_word_indices(sub_strings, data):
    finded_indices = []
    for i in sub_strings:
        matched_str = re.search(re.escape(i), data, re.IGNORECASE)
        finded_indices.append([matched_str.start(), matched_str.end()])

    return finded_indices

In [28]:
def find_kanuns(data):
    '''
    Funtion to find all Kanuns
    Input:
        data: an ictihat (string)

    Output:
        list of starting and ending indices of all finded Kanuns
    '''
    finded_kanuns = []
    start_idx = -1 # start index is initially -1 then update later
    splitted_data = data.split() # splitting an inctihat into words
    for idx in range(len(splitted_data)):
        try:
            # if word is an integer like 5237 and sayılı is next word set the start index
            int(splitted_data[idx])
            if splitted_data[idx+1].lower() == 'sayılı':
                start_idx = idx
        except:
            # if it is not int but starts with aynı and kanun set that as a starting index
            if splitted_data[idx].lower() == 'aynı' and "kanun" in splitted_data[idx+1].lower():
                start_idx = idx

        # end of kanun if starting index is not -1 and word includes madde
        if start_idx != -1 and 'madde' in splitted_data[idx].lower():
            finded_kanuns.append(" ".join(splitted_data[start_idx:idx+1]))
            start_idx = -1

    return find_word_indices(finded_kanuns, data)

In [29]:
def find_ust_mahkeme(data):
    '''
    Funtion to find all Ust Mahkemes
    Input:
        data: an ictihat (string)

    Output:
        list of starting and ending indices of all finded Ust Mahkemes
    '''
    finded_mahkemes = []

    splitted_data = data.split() # splitting an inctihat into words
    for idx, word in enumerate(splitted_data):
        # if word includes Mahkeme we are looking previous words till the begging of Mahkeme Names
        if 'Mahkeme' in word and word.lower() != 'mahkemece' and 'kanun' not in splitted_data[idx+1].lower():
            look_back = 1
            while True:
                starts_with = splitted_data[idx-look_back][0]
                try:
                    int(starts_with)
                    look_back += 1
                except:
                    # Usually all words of a Mahkeme name starts with capital letter
                    # So that we are looking back all words starts with capital letter
                    if starts_with.isupper():
                        look_back += 1
                    else:
                        # if it is not end the loop
                        look_back -= 1
                        break

            finded_mahkemes.append(" ".join(splitted_data[idx-look_back:idx+1]))

    cleaned_mahkemes = []
    for mahkeme in finded_mahkemes:
        if len(mahkeme.split()) == 1:
            continue
        else:
            cleaned_mahkemes.append(mahkeme)
    
    return find_word_indices(cleaned_mahkemes, data)

In [30]:
def isint(value):
    """
    Function to check if given value is an integer or not
    """
    try:
        int(value)
        return True
    except:
        return False

def find_dates(data):
    """
    Function to find dates
    """
    finded_dates = []

    splitted_data = data.split()
    for idx, word in enumerate(splitted_data):
        # if the lenght of a word is equal to 10 and can be splitted into three with ".", "/" that means it is a date
        if len(word) == 10:
            splitted_word = word.split('.')
            splitted_word2 = word.split('/')
            if len(splitted_word) == 3 and isint(splitted_word[0]) and isint(splitted_word[1]) and isint(splitted_word[2]):
                finded_dates.append(idx)
            elif len(splitted_word2) == 3 and isint(splitted_word2[0]) and isint(splitted_word2[1]) and isint(splitted_word2[2]):
                finded_dates.append(idx)

    return finded_dates

In [31]:
def create_df(kanun_indices, mahkeme_indices, date_indices, data):
    """
    Function to create a Named Entity Recognition DataFrame
    Input:
        kanun_indices: finded kanuns for a given ictihat
        mahkeme_indices: finded ust mahkemes for a given ictihat
        date_indices: finded dates
        data: an ictihat (string)
    Output:
        A dataframe has "words" and "labels" columns
        words: word
        labels: BIO tags
    """
    splitted_data = data.split()

    used_indices = []
    df = []
    for sample in kanun_indices:
        before_sample = data[:sample[0]].split()
        sample_data = data[sample[0]:sample[1]].split()

        used_indices += list(range(len(before_sample), len(before_sample) + len(sample_data)))

        df += [[sample_data[i], "I-KAN"] if i != 0 else [sample_data[i], "B-KAN"] for i in range(len(sample_data))]

    for sample in mahkeme_indices:
        before_sample = data[:sample[0]].split()
        sample_data = data[sample[0]:sample[1]].split()

        used_indices += list(range(len(before_sample), len(before_sample) + len(sample_data)))

        df += [[sample_data[i], "I-MAH"] if i != 0 else [sample_data[i], "B-MAH"] for i in range(len(sample_data))]

    for sample in date_indices:
        df.append([splitted_data[sample], "B-DATE"])
        used_indices.append(sample)

    used_indices = set(used_indices)

    # After BI tagging we are tagging all other words with "O"
    for idx in range(len(splitted_data)):
        if idx not in used_indices:
            df.append([splitted_data[idx], "O"])


    return pd.DataFrame(df, columns=['words', 'labels'])

### An Examples of Kanuns, Ust Mahkemes and Dates

In [32]:
finded_kanuns = find_kanuns(cleaned_jsons[1]['ictihat'])
finded_mahkemes = find_ust_mahkeme(cleaned_jsons[1]['ictihat'])
finded_dates = find_dates(cleaned_jsons[1]['ictihat'])

In [37]:
# example text
cleaned_jsons[1]['ictihat']

'(5237 S. K. m. 52, 62, 86) (5271 S. K. m. 231) Eşe karşı basit yaralama suçundan sanık ...\'ın, 5237 sayılı Türk Ceza Kanunu\'nun 86/2, 86/3-a, 62/1 ve 52/2. maddeleri uyarınca 3.000,00 Türk Lirası adlî para cezası ile cezalandırılmasına, 5271 sayılı Ceza Muhakemesi Kanunu\'nun 231/5. maddesi gereğince hükmün açıklanmasının geri bırakılmasına dair Kırıkkale 4. Asliye Ceza Mahkemesinin 07.11.2019 tarihli ve 2018/535 Esas, 2019/948 Karar sayılı kararına karşı yapılan itirazın kabulü ile anılan kararın kaldırılmasına ilişkin mercii Kırıkkale 1. Ağır Ceza Mahkemesinin 05.02.2020 tarihli ve 2020/139 değişik iş sayılı kararına karşı Adalet Bakanlığının 17.11.2020 tarihli ve 2020/9153 sayılı yazısıyla kanun yararına bozma isteminde bulunulduğundan bu işe ait dava dosyası Yargıtay Cumhuriyet Başsavcılığının 15.12.2020 tarihli ve 2020/105078 sayılı tebliğnamesi ile Dairemize gönderilmekle incelendi. Mezkur ihbarnamede; 5271 sayılı Kanun’un 231/8. maddesinde yer alan “Denetim süresi içinde, kiş

In [36]:
finded_kanuns

[[95, 165],
 [237, 290],
 [921, 959],
 [1882, 1914],
 [2017, 2052],
 [3079, 3114],
 [4079, 5005]]

In [35]:
for idx in range(len(finded_kanuns)):
    print(cleaned_jsons[1]['ictihat'][finded_kanuns[idx][0]:finded_kanuns[idx][1]])

5237 sayılı Türk Ceza Kanunu'nun 86/2, 86/3-a, 62/1 ve 52/2. maddeleri
5271 sayılı Ceza Muhakemesi Kanunu'nun 231/5. maddesi
5271 sayılı Kanun’un 231/8. maddesinde
5271 sayılı CMK'nin 309. maddesi
5271 sayılı CMK’nin 231. maddesinde
6545 sayılı Kanun’un 72. maddesinin
6545 sayılı Kanun'un yürürlüğe girdiği tarihten sonra işlenen suçlar için, hakkında daha önce hükmün açıklanmasının geri bırakılması kararı bulunan sanıklarla ilgili bir daha hükmün açıklanmasının geri bırakılması kararı verilemeyecektir. İnceleme konusu somut olayda; mahkemece sanık ...’ın kasten basit yaralama suçundan adli para cezası ile cezalandırılmasına ve hükmün açıklanmasının geri bırakılmasına karar verilmiştir. Bu karara karşı yapılan itiraz merciince kabul edilerek sanık hakkındaki hükmün açıklanmasının geri bırakılmasına dair karar kaldırılmıştır. Sanığın adli sicil kaydında yer alan kasten basit yaralama suçundan verilen Kırıkkale 3. Asliye Ceza Mahkemesinin 10.05.2018 tarihli ve 2017/523 Esas, 2018/315 Kara

In [38]:
finded_mahkemes

[[347, 384], [532, 567], [1351, 1388], [1351, 1388]]

In [39]:
for idx in range(len(finded_mahkemes)):
    print(cleaned_jsons[1]['ictihat'][finded_mahkemes[idx][0]:finded_mahkemes[idx][1]])

Kırıkkale 4. Asliye Ceza Mahkemesinin
Kırıkkale 1. Ağır Ceza Mahkemesinin
Kırıkkale 3. Asliye Ceza Mahkemesinin
Kırıkkale 3. Asliye Ceza Mahkemesinin


In [40]:
finded_dates

[57, 81, 92, 111, 171, 182, 197, 216, 591, 604, 697, 707, 711, 779]

In [42]:
for idx in range(len(finded_dates)):
    print(cleaned_jsons[1]['ictihat'].split()[finded_dates[idx]])

07.11.2019
05.02.2020
17.11.2020
15.12.2020
06.06.2017
10.05.2018
08.10.2018
21.05.2018
10.05.2018
08.10.2018
08.10.2018
07.11.2019
08.10.2018
04.01.2021


In [45]:
df = create_df(finded_kanuns, finded_mahkemes, finded_dates, cleaned_jsons[1]['ictihat'])

In [46]:
df['sentence_id'] = np.array([0] * df.shape[0])

In [48]:
# Example DataFrame for an ictihat
df = df[['sentence_id', 'words', 'labels']]
df

,sentence_id,words,labels
0,0,5237,B-KAN
1,0,sayılı,I-KAN
2,0,Türk,I-KAN
3,0,Ceza,I-KAN
4,0,Kanunu'nun,I-KAN
...,...,...,...
788,0,oybirliği,O
789,0,ile,O
790,0,karar,O
791,0,verildi.,O


# Preparing Dataset

In [49]:
tagged_data = [] # keeps dataframes

# tagging all ictihats
for idx, ictihat in enumerate(tqdm(cleaned_jsons)):
    finded_kanuns = find_kanuns(ictihat['ictihat'])
    finded_mahkemes = find_ust_mahkeme(ictihat['ictihat'])
    finded_dates = find_dates(ictihat['ictihat'])

    df = create_df(finded_kanuns, finded_mahkemes, finded_dates, ictihat['ictihat'])

    df['sentence_id'] = np.array([idx] * df.shape[0])
    df = df[['sentence_id', 'words', 'labels']]

    tagged_data.append(df)

100%|██████████| 27842/27842 [01:18<00:00, 355.60it/s]


In [50]:
len(tagged_data)

27842

In [51]:
# concatenating all tagged ictihats into a single DataFrame
combined_tagged_data = tagged_data.pop(0)
for idx in tqdm(list(range(0, len(tagged_data), 20))):
    combined_tagged_data = pd.concat([combined_tagged_data, *[tagged_data.pop(0) for i in range(len(tagged_data[idx:idx+20]))]], axis=0)

100%|██████████| 1393/1393 [02:09<00:00, 10.73it/s]


In [52]:
combined_tagged_data = combined_tagged_data.reset_index(drop=True)
combined_tagged_data['sentence_id'] = combined_tagged_data['sentence_id'].astype(int)
combined_tagged_data

,sentence_id,words,labels
0,0,5237,B-KAN
1,0,sayılı,I-KAN
2,0,Türk,I-KAN
3,0,Ceza,I-KAN
4,0,Kanunu'nun,I-KAN
...,...,...,...
3605056,13921,"ONANMASINA,",O
3605057,13921,tarihinde,O
3605058,13921,oybirliğiyle,O
3605059,13921,karar,O


In [53]:
combined_tagged_data.to_csv(f"{output_saving_path}/combined_tagged_data_for_ner.csv", index=False)